# 0. Prepare

In [3]:
import zipfile
import time
import pandas as pd
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt


from zipfile import ZipFile
from memory_profiler import profile

In [4]:
url = '/Users/thienhtt20/Documents/GitHub/Handling-Data/MyData/2019-Nov.csv.zip'

In [ ]:
# df = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"')
# df.info(verbose=False, memory_usage="deep")

In [ ]:
df = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"', nrows=10)

In [ ]:
df.columns.tolist()

# 1.Better read file

### 1.1.Basic Feature Selection For Analysis

1. As we can see, there are 5 features we actually need to do care about:

    [**'category_code', 'brand', 'price', 'event_time', 'event_type'** ]
2. The rest would be outliers and redundant due to its data format and unreadble

3. Data size is enormous ~60M records --> cannot read immediately
4. Data type unoptimized (**float64 -> float16**)

In [ ]:
df.head(2)

In [5]:
needed_categories = ['event_time', 'category_code', 'event_type', 'brand', 'price']

### 1.2 Optimized Data Type

#### 1.2.1 FLOAT TYPE

In [ ]:
df[needed_categories].dtypes

In [ ]:
df = df[needed_categories]

In [ ]:
df['price'] = df['price'].astype('float16')

#### 1.2.2 TIME TYPE

In [ ]:
df.head(4)

In [ ]:
df['event_time'] = df['event_time'].apply(lambda x: x.split(' ')[0])

In [ ]:
df['event_time'] = pd.to_datetime(df['event_time'])

In [ ]:
df['event_time'].dtypes

In [ ]:
df['event_time']

# 2. READ WITH CHUNK_SIZE

### 2.1 Chunk_size and check null

In [ ]:
df_test = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"', chunksize=1000000)

In [ ]:
for chunk in df_test:
    chunk = chunk[needed_categories]
    print(chunk.isna().sum())
    break

### 2.2 More Details

- From 1M records with each chunk, we are able to reduce ~40% records from 1M to roughly 600K records for the **FIRST** chunk

In [5]:
import pandas as pd

chunk_size = 1000000
df_chunks = pd.read_csv(url, compression='zip', header=0, sep=',', quotechar='"', chunksize=chunk_size)


for i, chunk in enumerate(df_chunks):
    print(f"Memory usage for chunk {i + 1}: {chunk.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")



Memory usage for chunk 1: 370.82 MB
Memory usage for chunk 2: 370.58 MB
Memory usage for chunk 3: 370.33 MB
Memory usage for chunk 4: 371.12 MB
Memory usage for chunk 5: 370.52 MB
Memory usage for chunk 6: 370.59 MB
Memory usage for chunk 7: 369.93 MB


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
df_chunks = pd.read_csv(url, compression='zip', header=0, 
                 sep=',', quotechar='"',
                 usecols=needed_categories, chunksize=1000000)
for i, chunk in enumerate(df_chunks):
    print(f"Memory usage for chunk {i + 1}: {chunk.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")


In [6]:
def process_data(url, chunk_size, needed_categories):
    start_time = time.time()
    df_chunks = pd.read_csv(url, compression='zip', header=0, 
                 sep=',', quotechar='"',
                 usecols=needed_categories, chunksize=chunk_size)
    large = []
    small = []
    final_output = pd.DataFrame()
    
    for i, chunk in enumerate(df_chunks):
        print(f"Memory usage for chunk {i}: {chunk.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
        large.append(chunk.memory_usage(deep=True).sum() / (1024 ** 2))

        chunk['count'] = np.int32(1)
        chunk['event_time'] = pd.to_datetime(chunk['event_time']).dt.date
        processed_chunk = chunk.groupby(needed_categories, as_index=False).agg({'count': 'sum'})
        processed_chunk['price'] = processed_chunk['price'].astype('float16')
        final_output = pd.concat([final_output, processed_chunk], ignore_index=True, sort=False)
        
        print(f"Memory usage after processing chunk {i}: {processed_chunk.memory_usage(deep=True).sum() / (1024 ** 2):.2f} MB")
        small.append(processed_chunk.memory_usage(deep=True).sum() / (1024 ** 2))
        
        print("-"*50)
        print('\n')

    final_output.to_csv("final_output.csv", index=False)
    end_time = time.time()
    print(f"Total time taken: {end_time - start_time:.2f} seconds")
    return final_output, large, small

In [8]:
final_output, large, small = process_data(url, chunk_size=1000000, 
                            needed_categories=needed_categories)

Memory usage for chunk 0: 259.24 MB
Memory usage after processing chunk 0: 6.18 MB
--------------------------------------------------


Memory usage for chunk 1: 259.00 MB
Memory usage after processing chunk 1: 9.25 MB
--------------------------------------------------


Memory usage for chunk 2: 258.75 MB
Memory usage after processing chunk 2: 5.92 MB
--------------------------------------------------


Memory usage for chunk 3: 259.54 MB
Memory usage after processing chunk 3: 5.91 MB
--------------------------------------------------


Memory usage for chunk 4: 258.94 MB
Memory usage after processing chunk 4: 9.04 MB
--------------------------------------------------


Memory usage for chunk 5: 259.01 MB
Memory usage after processing chunk 5: 6.11 MB
--------------------------------------------------


Memory usage for chunk 6: 258.35 MB
Memory usage after processing chunk 6: 8.97 MB
--------------------------------------------------


Memory usage for chunk 7: 258.33 MB
Memory usage